# Crypto 1 - Baby Feistel

> I made my own custom cipher but somehow encrypting twice does nothing.
> 
> - ariana
>
> Files: [main.py](main.py)

The challenge is a pretty basic Feistel cipher, except it is demonstrated that encrypting any text twice returns itself, or equivalently that encryption is identical to decryption. We are also given that each sub-key $K_{i+1}$ is derived directly from the previous sub-key $K_{i}$.

From the Feistel diagram (from Wikipedia), this implies that the key schedule must be palindromic.

![schedule](Feistel_cipher_diagram_en.svg)

The easiest way for this to happen is for every sub-key to be same, so let's assume this hypothesis for now. In that case, the relevant code that derives $K_{i+1}$ from $K_i$ is given by:

```py
key = sum(M*pow(key,n,5**55) for n,M in enumerate(MIX))%(5**55)
```

which is basically a polynomial of degree 16 (since `len(MIX) == 17`).

We'll call this polynomial $f$, and it remains to just solve $f(x) \equiv x\pmod {5^{55}}$. We can treat $x$ and a base-5 number and brute-force the values from right to left. This is equivalent to first solving it $\bmod{5}$, then $\bmod{25}$, then $\bmod{125}$ and so on.

Of course, the solution may not be unique, so for each valid solution mod $5^k$ we can lift it to solutions modulo higher powers. Then we just look for a value that's entirely ASCII.

In [1]:
from Crypto.Util.number import *

MIX = [107632798521572819694764108404904109375, 41536741957891467431472232877751864376, 275576770983938755827220589369303781975, 234236451662009148302826684340536433335, 167356592051306863969732200182719434359, 3964872179755557916019346905353137736, 261654592197979525204490051641223914986, 130534506910667985170179545870071095171, 110547794064929106567581040448523263944, 184063535252587585286044863527738743538, 164357339419512390377522630828102169288, 199930288846991086238436587886041223038, 177192341130030822677616530029496840095, 214572440504728654991188419790259127601, 114580049840189248087547627033034844626, 160013292840662875257096750676572257331, 27182818284590452353602874713526624977]

F.<x> = ZZ[]
f = F(MIX)

def lift(i, mod):
    if f(i) % mod == i:
        if mod == 5^55:
            yield long_to_bytes(i)
        else:
            for j in range(5):
                yield from lift(i + mod * j, mod * 5)
            
for bs in lift(0, 1):
    if all(x < 128 for x in bs):
        print(bs)

b'vuln_f1x3d_p01n7'


The flag is `grey{vuln_f1x3d_p01n7}`.